In [50]:
# 구글 코랩 환경에서 실행, 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [51]:
# 구글 드라이브 경로 지정
path = '/content/drive/My Drive/Colab Notebooks/2020-PoscoICT/Data/'

In [ ]:
import hanja

In [ ]:
# 한자 변환
!pip install hanja

# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu, mac 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 한글 여부 판단
!pip install soynlp


### 전처리
1. 파일 이름은 한국어가 아닌 영어로 지정하기.
2. csv 파일의 데이터 column name은 영어로 지정하기.
3. 특수문자 ▶,ㅁ,】 등은 되도록이면 사용하지 않기.
4. 같은 파일내 데이터 저장 포맷은 통일하고, 데이터 전처리 모듈을 각각 따로 구현하기.
5. 모델 학습 또는 평가 데이터로 사용할 데이터는 따로 파일로 저장하기
    - 파일 저장할 때, 문장 split으로 "\u241E" 또는 "\u241D" 추천

In [ ]:
import re

removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

EMAIL_PATTERN = re.compile(r'''(([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4}))''', re.VERBOSE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.VERBOSE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

In [ ]:
def cleansing_other(sentence: str = None):
    """
    문장을 전처리 (이메일, URL, 공백 등 제거) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub(EMAIL_PATTERN, ' ', sentence)
    sentence = re.sub(URL_PATTERN, ' ', sentence)
    sentence = re.sub(MULTIPLE_SPACES, ' ', sentence)
    sentence = sentence.replace(", )", "")
    
    return sentence

In [ ]:
def cleansing_chinese(sentence: str = None) -> str:
    """
    한자를 변환하는 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    # chinese character를 앞뒤로 괄호가 감싸고 있을 경우, 대부분 한글 번역임
    sentence = re.sub("\([\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]+\)", "", sentence)
    # 다른 한자가 있다면 한글로 치환
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sentence) is not None:
        sentence = hanja.translate(sentence, 'substitution')

    return sentence

In [ ]:
def cleansing_numbers(sentence: str = None) -> str:
    """
    숫자를 전처리(delexicalization) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

In [ ]:
def cleansing_special(sentence: str = None) -> str:
    """
    특수문자를 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    # print(sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

In [ ]:
def preprocess_sent(sentence: str = None) -> str:
    """
    모든 전처리를 수행 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sent_clean = sentence
    sent_clean = cleansing_other(sent_clean)
    sent_clean = cleansing_chinese(sent_clean)
    sent_clean = cleansing_special(sent_clean)
    sent_clean = cleansing_numbers(sent_clean)
    sent_clean = re.sub('\s+', ' ', sent_clean)

    return sent_clean

In [ ]:
def post_process(tokens):
    results = []
    for token in tokens:
        # 숫자에 공백을 주어서 띄우기
        # 이외 추가로 도메인에 맞게 post preprocessing을 할 수 있음.
        processed_token = [el for el in re.sub(r"(\d)", r" \1 ", token).split(" ") if len(el) > 0]
        results.extend(processed_token)
    return results

### 사용가능한 형태소 분석기 

1. 카카오 형태소 분석기 설치 가이드
- https://provia.tistory.com/56
2. KoNLPy의 Okt, Komoran, Mecab, Hannanum KKma
- https://konlpy.org/ko/v0.5.2/api/konlpy.tag/#module-konlpy.tag._kkma
3. Soynlp
- https://github.com/lovit/soynlp

In [ ]:
# from khaiii import KhaiiiApi
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

In [ ]:
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    elif tokenizer_name == "khaiii":
        tokenizer = KhaiiiApi()
    else:
        tokenizer = Mecab()
    return tokenizer

In [ ]:
def tokenize(tokenizer_name, original_sent, pos=False):
    tokenizer = get_tokenizer(tokenizer_name)
    sentence = original_sent.replace('\n', '').strip()
    if tokenizer_name == "khaiii":
        tokens = []
        for word in tokenizer.analyze(sentence):
            if pos:
                tokens.extend([str(m) for m in word.morphs])
            else:
                tokens.extend([str(m).split("/")[0] for m in word.morphs])
    else:
        if pos:
            tokens = tokenizer.pos(sentence)
            tokens = [morph + "/" + tag for morph, tag in tokens]
        else:
            tokens = tokenizer.morphs(sentence)
    tokenized_sent = ' '.join(post_process(tokens))
    
    return tokenized_sent


#### 참고
- 한국어 자연어처리에서 자주 사용하는 문장분리(아래 코드 참고)
- http://docs.likejazz.com/kss/

In [ ]:
sample = "Q1. 일단 이 로비 의혹까지 불거진 열차제어시스템이라는 기술이 아주 중요한 거라면서요? 쉽게 말해서 열차 운행에서의 2중 안전장치라고 생각하시면 될 것 같습니다. 보통 열차는 기관사가 운행 통제를 하는데, 기관사가 잘못 조작을 할 경우엔 대규모 인명피해로 이어질 수 있습니다."

In [ ]:
import re
sentences = re.split("(?<=[.!?])\s+", sample.strip())
print(sentences)

['Q1.', '일단 이 로비 의혹까지 불거진 열차제어시스템이라는 기술이 아주 중요한 거라면서요?', '쉽게 말해서 열차 운행에서의 2중 안전장치라고 생각하시면 될 것 같습니다.', '보통 열차는 기관사가 운행 통제를 하는데, 기관사가 잘못 조작을 할 경우엔 대규모 인명피해로 이어질 수 있습니다.']


In [ ]:
tokenize("komoran", sample)

'Q 1 . 일단 이 로비 의혹 까지 불거지 ㄴ 열차 제어 시스템 이 라는 기술 이 아주 중요 하 ㄴ 거 이 라면서요 ? 쉽 게 말 하 아서 열차 운행 에서 의 2 중 안전장치 이 라고 생각 하 시 면 되 ㄹ 것 같 습니다 . 보통 열차 는 기관사 가 운행 통제 를 하 는데 , 기관사 가 잘못 조작 을 하 ㄹ 경우 에 ㄴ 대 규모 인명 피해 로 이어지 ㄹ 수 있 습니다 .'

In [ ]:
!pip install kss
import kss

  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251523 sha256=39d9b23bffccabc85817b525cde691c6a8e41be94f6017e4f248ba9ead777ae0
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
Successfully built kss


In [ ]:
result = kss.split_sentences(sample)
print(result)

['Q1. 일단 이 로비 의혹까지 불거진 열차제어시스템이라는 기술이 아주 중요한 거라면서요?', '쉽게 말해서 열차 운행에서의 2중 안전장치라고 생각하시면 될 것 같습니다.', '보통 열차는 기관사가 운행 통제를 하는데, 기관사가 잘못 조작을 할 경우엔 대규모 인명피해로 이어질 수 있습니다.']


In [ ]:
sentence = '아버지가방에들어가신다'

In [ ]:
# Okt
print("=== Okt ===")
print(tokenize("okt", sentence))

# Komoran
print("=== Komoran ===")
print(tokenize("komoran", sentence))

# Mecab
print("=== Mecab ===")
print(tokenize("mecab", sentence))

# Hannanum
print("=== Hannanum ===")
print(tokenize("hannanum", sentence))

# Hannanum
print("=== Kkma ===")
print(tokenize("kkma", sentence))


=== Okt ===
아버지 가방 에 들어가신다
=== Komoran ===
아버지 가방 에 들어가 시 ㄴ다
=== Mecab ===
아버지 가 방 에 들어가 신다
=== Hannanum ===
아버지가방에들어가 이 시ㄴ다
=== Kkma ===
아버지 가방 에 들어가 시 ㄴ다


### 연습

In [ ]:
sample_df = pd.read_csv('nsmc/ratings_train.txt', sep='\t')
print(sample_df.shape)
print(sample_df.columns)

In [ ]:
corpus = sample_df['document'].values
print(len(corpus))
print(corpus[:10])

- ratings_test.txt 활용
- 전체 데이터 중에 positive label(=1)만 사용
- 데이터 문장 중 앞에서 1000개 문장만 활용
    - pandas dataframe의 head(1000)활용

In [ ]:
import pandas as pd

In [ ]:
sample_df = pd.read_csv(path+'nsmc/ratings_test.txt', sep='\t')
print(sample_df.shape)

In [ ]:
positive_df = sample_df[sample_df.label==1].head(1000)
# loc iloc

In [ ]:
positive_df.head(10)

In [ ]:
all_sent_df = positive_df[['id','document']]
print(all_sent_df.shape)

(1000, 2)


In [ ]:
all_sent_df['tokenized'] = all_sent_df['document'].map(lambda x: tokenize("mecab", preprocess_sent(x)))

          id  ...                                          tokenized
0    6270596  ...                                                  굳
5    7898805  ...                              음악 이 주 가 된 최고 의 음악 영화
9    6242223  ...  이별 의 아픔 뒤 에 찾아오 는 새로운 인연 의 기쁨 But 모든 사람 이 그렇 지...
10   7462111  ...                              괜찮 네요 오랜만 포켓몬스터 잼 밌어요
12   6900881  ...  청춘 은 아름답 다 그 아름다움 은 이성 을 흔들 어 놓 는다 찰나 의 아름다움 을...
13   9629375  ...          눈 에 보이 는 반전 이 었 지만 영화 의 흡인력 은 사라지 지 않 았 다
15  10268521  ...                                  소위 문가 라는 평점 은 뭐 냐
16   2968565  ...                                                 최고
18   6406912  ...                                                나이스
20   9305768  ...  NUM 일 의 금요일 나이트메어 시리즈 와 함께 가장 많 은 시리즈 를 양산 해냈 ...

[10 rows x 3 columns]


In [ ]:
all_sent_df['tokenized'].values

In [ ]:
def save_file(sent_list, filename):
  with open(filename,'w',encoding='utf-8') as f:
    for sent in sent_list:
      f.write(sent+'\n')

In [52]:
save_file(all_sent_df['tokenized'].values, path+'ratings_test_tokenized.txt')